In [1]:
from data.overnight.domain.domain_base import Domain
import re
import tempfile
import subprocess

In [2]:
overnight_domains = ['basketball', 'blocks', 'calendar', 'housing', 'publications', 'recipes', 'restaurants', 'socialnetwork']

In [48]:
def read_data(path, domain_idx):
    ex_list = []
    with open(path, 'r') as infile:
        lines = [line.strip() for line in infile]
        for i in range(len(lines))[::8]:
            ex = {}
            ex['q'] = re.findall(r'''\"(.*?)\"''',lines[i+1])[0]
            ex['tir'] = re.findall(r'''\"(.*?)\"''',lines[i+2])[0]
            ex['lamb'] = lines[i+4].strip()
            ex['domain'] = domain_idx
            ex_list.append(ex)
    return ex_list

In [49]:
ex_list = []
for idx, domain in enumerate(overnight_domains):
    ex_list += read_data('../sempre/lib/data/overnight/{}.paraphrases.test.examples'.format(domain), idx)
# test_inputs = [i['q'] for i in ex_list]
# test_targets = [i['tir'] for i in ex_list]
# test_lambs = [i['lamb'] for i in ex_list]
test_domains = [i['domain'] for i in ex_list]

In [45]:
test_preds = []
test_golds = []
with open("./lambda_results.txt") as f:
    for line in f:
        test_preds.append(line.strip())
with open("./lambda_golds.txt") as f:
    for line in f:
        test_golds.append(line.strip())

In [11]:
def read_data(path):
    ex_list = []
    with open(path, 'r') as infile:
        for line in infile:
            line = line.strip()
            if line == '':
                continue
            q, lf = line.split('\t')
            ex_list.append({"q": q.strip(), "lf": lf.strip()})
    return ex_list

In [14]:
test_gold = []
for domain in overnight_domains:
    test_gold += [i['lf'] for i in read_data("./data/overnight/dataset/{}_test.tsv".format(domain))]

In [57]:
wrong_pairs = []
count = 0
assert len(test_domains) == len(test_preds) == len(test_golds) == len(test_gold)
for p, g, i, d in zip(test_preds, test_golds, test_inputs, test_domains):
    # p = p.replace("!", " !").replace(".size", " .size")
    if p != g:
        print(len(wrong_pairs))
        wrong_pairs.append((d, [p,g]))
        print("input:\t", i)   
        print("pred:\t", p)
        print("gold:\t", g)
        print("\n")
# print("wrong_pairs:", len(wrong_pairs))
# print("acc:", 1 - len(wrong_pairs) / len(test_preds))

0
input:	 name a player with more than 3 rebounds in an average season
pred:	 ( call SW.listValue ( call SW.getProperty ( ( lambda s ( call SW.filter ( var s ) ( call SW.ensureNumericProperty ( string num_rebounds ) ) ( string > ) ( call SW.ensureNumericEntity ( number 3 rebound ) ) ) ) ( call SW.domain ( string player ) ) ) ( string player ) ) )
gold:	 ( call SW.listValue ( call SW.getProperty ( ( lambda s ( call SW.filter ( var s ) ( call SW.ensureNumericProperty ( string num_rebounds ) ) ( string >= ) ( call SW.ensureNumericEntity ( number 3 rebound ) ) ) ) ( call SW.domain ( string player ) ) ) ( string player ) ) )


1
input:	 find the player that has the hightest point score over any one season
pred:	 ( call SW.listValue ( call SW.getProperty ( ( lambda s ( call SW.superlative ( var s ) ( string min ) ( call SW.ensureNumericProperty ( string num_points ) ) ) ) ( call SW.domain ( string player ) ) ) ( string player ) ) )
gold:	 ( call SW.listValue ( call SW.getProperty ( ( lambda 

In [86]:
def format_overnight(lf):
    replacements = [
        ('(', ' ( '), # make sure ( and ) must have blank space around
        (')', ' ) '),
        ('! ', '!'),
        ('SW', 'edu.stanford.nlp.sempre.overnight.SimpleWorld'),
    ]
    for a, b in replacements:
        lf = lf.replace(a, b)
    # remove redundant blank spaces
    lf = re.sub(' +', ' ', lf)
    return lf.strip()

def normalize(lf_list):
    return [format_overnight(lf) for lf in lf_list]

def obtain_denotations(lf_list, domain):
    tf = tempfile.NamedTemporaryFile('w+t', encoding='utf8', suffix='.examples')
    for line in lf_list:
        tf.write(line + '\n')
    tf.flush()
    msg = subprocess.check_output(['./evaluator/overnight', domain, tf.name])
    msg = msg.decode('utf8')
    tf.close()
    denotations = [
        line.split('\t')[1] for line in msg.split('\n')
        if line.startswith('targetValue\t')
    ]
    return denotations

In [13]:
def is_valid(ans):
    return not ('BADJAVA' in ans or 'ERROR' in ans or 'Exception' in ans or ans == 'null')

In [71]:
normalize(x[1])

['( call edu.stanford.nlp.sempre.overnight.SimpleWorld.listValue ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.filter ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.filter ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.getProperty ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.singleton en.restaurant ) ( string! type ) ) ( string delivery ) ) ( string price_rating ) ( string = ) ( number 2 en.dollar_sign ) ) )',
 '( call edu.stanford.nlp.sempre.overnight.SimpleWorld.listValue ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.filter ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.filter ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.getProperty ( call edu.stanford.nlp.sempre.overnight.SimpleWorld.singleton en.restaurant ) ( string! type ) ) ( string price_rating ) ( string = ) ( number 2 en.dollar_sign ) ) ( string delivery ) ) )']

In [90]:
x = wrong_pairs[505]
ans = obtain_denotations(normalize(x[1]), overnight_domains[x[0]])
print(ans[0])
print(ans[1])

null
null


java.lang.RuntimeException: Invalid number of arguments for join (want 2): (string!type)
	at edu.stanford.nlp.sempre.Formulas.fromLispTree(Formulas.java:80)
	at edu.stanford.nlp.sempre.Formulas.fromLispTree(Formulas.java:41)
	at edu.stanford.nlp.sempre.Formulas.fromLispTree(Formulas.java:41)
	at edu.stanford.nlp.sempre.Formulas.fromLispTree(Formulas.java:41)
	at edu.stanford.nlp.sempre.Formulas.fromLispTree(Formulas.java:41)
	at edu.stanford.nlp.sempre.overnight.ExecuteLogicalForms.readFormulas(ExecuteLogicalForms.java:27)
	at edu.stanford.nlp.sempre.overnight.ExecuteLogicalForms.run(ExecuteLogicalForms.java:41)
	at fig.exec.Execution.runWithObjArray(Execution.java:337)
	at fig.exec.Execution.run(Execution.java:325)
	at edu.stanford.nlp.sempre.overnight.ExecuteLogicalForms.main(ExecuteLogicalForms.java:57)
java.lang.RuntimeException: Invalid number of arguments for join (want 2): (string!type)
	at edu.stanford.nlp.sempre.Formulas.fromLispTree(Formulas.java:80)
	at edu.stanford.nlp.semp